### Title: Midterm Programming question Part2
### Author: Sai Vinay Thattukolla
### Date: 10/4/2023

The project is a community chat system extension that will add new functionality to an existing chat system that currently suppports 75 million users. this new file feature allow the users to include an attached file with their messages. our proposed system should keep this file with fields user_id, date, file type, and the file itself. for the sake of our database, i took 3 more fields in my database fields. those are filename, file location and content in the file. In reality content in each file is masked to the database adminsitrators for securoty reasons.


In the below code i am creating different types of file ssuch as png, pdf, jpg and doc with unique names for each files. I also inserted the content in each file with the help of write function using python.

In [83]:
with open('file11.jpg','w') as fout:
    fout.write("this is user 11")

with open('file12.doc','w') as fout1:
    fout1.write("this is user 12")

with open('file13.pdf','w') as fout2:
    fout2.write("this is user 13")

with open('file14.mp4','w') as fout3:
    fout3.write("this is user14")

with open('file15.png','w') as fout4:
    fout4.write("this is user15")

After creating the files, our next step is to read the files. so i am utliizing the read function in python that reads the content in the each document.

In [84]:
with open('file11.jpg', 'r') as fin1:
    tmp1=fin1.read()
    print(tmp1)

with open('file12.doc', 'r') as fin2:
    tmp2=fin2.read()
    print(tmp2)

with open('file13.pdf', 'r') as fin3:
    tmp3=fin3.read()
    print(tmp3)

with open('file14.mp4', 'r') as fin4:
    tmp4=fin4.read()
    print(tmp4)

with open('file15.png', 'r') as fin5:
    tmp5=fin5.read()
    print(tmp5)

this is user 11
this is user 12
this is user 13
this is user14
this is user15


Here i am importing the required libraries. using 
1. pandas as pd 
2. importing the cassandra cluster. 

In the terminal, i have started the cassandra cluster using the "docker-compose up". this will start the 3-node cassandra cluster. To start the cassandra cluster, we need docker.yaml file. it contains the instructiins to start the docker cluster with mentioned ports and addresses. (check the .yaml file for more information).

In [85]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [86]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

Before proceeding with the database insertion and mnaipulation, it is important for us to create key space. it is the outermost region in the cassandra cluster. 

In [87]:
session.execute("DROP KEYSPACE IF EXISTS w04python")

In the bwlow code we are checking the keyspaces currently present in the environment.

In [88]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In the below code, we are cretaing a key space with replication factor =3, which states data is replicated in total of 3 ndoes in 3 casandra cluster. which includes 1 master copy and 2 back up copies in differnt nodes.

In [89]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS w04python WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [90]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Create a each user file sharing dataset

In the below code we are creating the table skeleton with the fields and data type for each field. I have taken one user_id as primary key. we can specify other fields as well. Now the partiton of data is going to be taken based on user_id. if we have given filetype as other primary key, it becomes clustering key, which is sued for sorting the data in each node.

In [91]:
session.execute("""
CREATE TABLE IF NOT EXISTS w04python.files ( 
    user_id INT, 
    date TEXT, 
    file_type TEXT, 
    file_location TEXT,
    file TEXT,
    content TEXT, 
    PRIMARY KEY(user_id)
);
""")

In [92]:
df = pd.read_csv('M01.csv')
df

,user_id,date,file_type,file_location,file
0,1000,'01/01/2023','jpg','/home/student/Workbench/bd-f23/bd-f23/file11....,'file11'
1,1001,'01/02/2023','mp4','/home/student/Workbench/bd-f23/bd-f23/file12....,'file12'
2,1002,'01/03/2023','jpg','/home/student/Workbench/bd-f23/bd-f23/file13....,'file13'
3,1004,'01/04/2023','png','/home/student/Workbench/bd-f23/bd-f23/file14....,'file14'
4,1005,'02/05/2023','pdf','/home/student/Workbench/bd-f23/bd-f23/file15....,'file15'


In [93]:
df['content']=[tmp1,tmp2,tmp3,tmp4,tmp5]

In [94]:
df

,user_id,date,file_type,file_location,file,content
0,1000,'01/01/2023','jpg','/home/student/Workbench/bd-f23/bd-f23/file11....,'file11',this is user 11
1,1001,'01/02/2023','mp4','/home/student/Workbench/bd-f23/bd-f23/file12....,'file12',this is user 12
2,1002,'01/03/2023','jpg','/home/student/Workbench/bd-f23/bd-f23/file13....,'file13',this is user 13
3,1004,'01/04/2023','png','/home/student/Workbench/bd-f23/bd-f23/file14....,'file14',this is user14
4,1005,'02/05/2023','pdf','/home/student/Workbench/bd-f23/bd-f23/file15....,'file15',this is user15


In [95]:
df.dtypes

user_id           int64
date             object
file_type        object
file_location    object
file             object
content          object
dtype: object

In [96]:
df.columns

Index(['user_id', 'date', 'file_type', 'file_location', 'file', 'content'], dtype='object')

#### Checking the records in the database with the fields mentioned

In [97]:
for index, row in df.iterrows():
    print(f"user_id = {row.user_id}, date ={row.date}, file_type = {row.file_type}, file_location = {row.file_location}, file = {row.file}, content = {row.content}")

user_id = 1000, date ='01/01/2023', file_type = 'jpg', file_location = '/home/student/Workbench/bd-f23/bd-f23/file11.jpg', file = 'file11', content = this is user 11
user_id = 1001, date ='01/02/2023', file_type = 'mp4', file_location = '/home/student/Workbench/bd-f23/bd-f23/file12.doc', file = 'file12', content = this is user 12
user_id = 1002, date ='01/03/2023', file_type = 'jpg', file_location = '/home/student/Workbench/bd-f23/bd-f23/file13.pdf', file = 'file13', content = this is user 13
user_id = 1004, date ='01/04/2023', file_type = 'png', file_location = '/home/student/Workbench/bd-f23/bd-f23/file14.mp4', file = 'file14', content = this is user14
user_id = 1005, date ='02/05/2023', file_type = 'pdf', file_location = '/home/student/Workbench/bd-f23/bd-f23/file15.png', file = 'file15', content = this is user15


In the below code we are inserting the records into the files database using the session.execute function. this will execute the CQL query inside it. it will iterate theough the each row and insert each record one by one into the database.

In [98]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]}, {row[5]});
        """
    )
    session.execute(f"""
        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]}, {row[4]}, '{row[5]}');
        """
    )


        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES (1000, '01/01/2023', 'jpg', '/home/student/Workbench/bd-f23/bd-f23/file11.jpg', 'file11', this is user 11);
        

        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES (1001, '01/02/2023', 'mp4', '/home/student/Workbench/bd-f23/bd-f23/file12.doc', 'file12', this is user 12);
        

        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES (1002, '01/03/2023', 'jpg', '/home/student/Workbench/bd-f23/bd-f23/file13.pdf', 'file13', this is user 13);
        

        INSERT INTO w04python.files (user_id, date, file_type, file_location, file, content)     
        VALUES (1004, '01/04/2023', 'png', '/home/student/Workbench/bd-f23/bd-f23/file14.mp4', 'file14', this is user14);
        

        INSERT INTO w04python.files (user_id, date, file_type, file_location

Now we are searching for one file and returing the file to thee user. whenever the user wants a message he can use indexing method to see the file content and filetype.;

In [99]:
# To search for and return attachment

rows = session.execute("select (user_id, date, file, content) from w04python.files where user_id = 1004")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, attachment={row[0][2]}, {row[0][3]}")

1004, 01/04/2023, attachment=file14, this is user14


Now we are deleting the record to check our cassandra system is working correct or not.

In [100]:
# delete the record
session.execute(f"""
    DELETE FROM w04python.files
    WHERE
        user_id = 1004;
    """
)

Once our work is done, it is important for the user to stop the cassandra cluster and remove the cluster from the environment using the docker stop and docker rm command.